In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime

data = pd.read_csv('housing_price_dataset.csv')
current_year = datetime.now().year
data["Age"] = current_year - data["YearBuilt"]
data_encoded = pd.get_dummies(data, columns=["Neighborhood"], drop_first=True, prefix='N')
data_encoded.head()

,SquareFeet,Bedrooms,Bathrooms,YearBuilt,Price,Age,N_Suburb,N_Urban
0,2126,4,1,1969,215355.283618,54,False,False
1,2459,3,2,1980,195014.221626,43,False,False
2,1860,2,1,1970,306891.012076,53,True,False
3,2294,2,1,1996,206786.787153,27,False,True
4,2130,5,2,2001,272436.239065,22,True,False


In [4]:
X = data_encoded.drop(["Price"], axis=1)
y = data_encoded["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {'n_neighbors': [1, 5, 9, 19, 49, 99, 149, 199, 223, 249, 299]}

knn = KNeighborsRegressor()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

best_n_neighbors = grid_search.best_params_['n_neighbors']

best_knn = KNeighborsRegressor(n_neighbors=best_n_neighbors)
best_knn.fit(X_train_scaled, y_train)

y_pred = best_knn.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = best_knn.score(X_test_scaled, y_test)

print(f"Best Hyperparameter (n_neighbors): {best_n_neighbors}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Best Hyperparameter (n_neighbors): 49
Mean Squared Error: 2525490475.294873
R-squared: 0.5600155060471426
